## Google Trends

In [5]:
from pytrends.request import TrendReq
from pytrends import dailydata
import pandas as pd
import time

### Creating the list of keywords

In [6]:
# List of keywords will be here - can create a loop for this as well
#SOURCE: https://stockanalysis.com/list/dow-jones-stocks/
djia_tickers = [

 'CRM',
 'KO',
 'MCD', 
 'CSCO', 
 'WBA'] 


In [7]:
djia_names = [
    "Microsoft",
    "Apple",
    #"Visa Inc.",
    "JPMorgan Chase & Co.",
    "UnitedHealth Group Incorporated",
    "Walmart Inc.",
    "Johnson & Johnson",
    "The Procter & Gamble Company",
    "The Home Depot, Inc.",
    "Merck & Co., Inc.",
    "Chevron Corporation",
    "Salesforce, Inc.",
    "The Coca-Cola Company",
    "McDonald's Corporation",
    "Cisco Systems, Inc.",
    "The Walt Disney Company",
    "Intel Corporation",
    "International Business Machines Corporation",
    "Verizon Communications Inc.",
    "Caterpillar Inc.",
    "NIKE, Inc.",
    "Amgen Inc.",
    "American Express Company",
    "Honeywell International Inc.",
    "The Boeing Company",
    "The Goldman Sachs Group, Inc.",
    "3M Company",
    "The Travelers Companies, Inc.",
    "Dow Inc.",
    "Walgreens Boots Alliance, Inc."
]

In [8]:
len(djia_names)

29

### Creating a function to call Google Trends for list of keywords

In [9]:
def google_trends(inputlist, start_year):
    combined_df = pd.DataFrame()  # Empty DataFrame to store combined data
    list_name = [name for name, value in globals().items() if value is inputlist][0]

    for string in inputlist:
            # Define the file name with .csv extension
        file_name = f"{string}.csv"
        
        retries = 0
        max_retries = 5
        while retries < max_retries:
            try:
                # Make the request to Google Trends API
                data = dailydata.get_daily_data(string, start_year, 1, 2023, 1, geo='US').reset_index()
                cols = ['date', f'{string}']
                data[cols].to_csv(file_name, index=False) #we only need the two columns 
                
                #print(f"CSV file '{file_name}' has been created.")
                #df = pd.read_csv(file_name, index_col='date')
                
                # Append the DataFrame to the combined DataFrame
                #combined_df = pd.concat([combined_df, df], axis=1)
                #combined_df = pd.merge(combined_df, df, left_index=True, right_index=True)

                break  # Exit the retry loop if successful
            except Exception as e:
                if "429" in str(e):
                    # Backoff strategy
                    wait_time = 2**retries
                    print(f"Too many requests. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                    retries += 1
                else:
                    print(f"An error occurred: {e}")
                    break  # Exit the retry loop if it's not a 429 error
        else:
            print("Max retries reached. Unable to fetch data.")


    #file_name = inputlist.csv
    combined_file_name = f"{list_name}.csv"
    combined_df.to_csv(combined_file_name, index=True)
    #print(f"Combined CSV file '{combined_file_name}' has been created.")

    return f"Combined CSV file '{combined_file_name}' has been created."


In [10]:
def google_trends_2(inputlist, start_year):
    combined_df = pd.DataFrame()  # Empty DataFrame to store combined data
    list_name = [name for name, value in globals().items() if value is inputlist][0]

    for string in inputlist:
            # Define the file name with .csv extension
        file_name = f"{string}.csv"
        
        retries = 0
        max_retries = 5
        while retries < max_retries:
            try:
                # Make the request to Google Trends API
                data = dailydata.get_daily_data(string, start_year, 1, 2023, 1, geo='US').reset_index()
                cols = ['date', f'{string}']
                data[cols].to_csv(file_name, index=False) #we only need the two columns 
                
                print(f"CSV file '{file_name}' has been created.")
                #df = pd.read_csv(file_name, index_col='date')
                
                # Append the DataFrame to the combined DataFrame
                #combined_df = pd.concat([combined_df, df], axis=1)
                #combined_df = pd.merge(combined_df, df, left_index=True, right_index=True)

                break  # Exit the retry loop if successful
            except Exception as e:
                if "429" in str(e):
                    # Backoff strategy
                    wait_time = 2**retries
                    print(f"Too many requests. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                    retries += 1
                else:
                    print(f"An error occurred: {e}")
                    break  # Exit the retry loop if it's not a 429 error
        else:
            print("Max retries reached. Unable to fetch data.")


    #file_name = inputlist.csv
    combined_file_name = f"{list_name}.csv"
    combined_df.to_csv(combined_file_name, index=True)
    #print(f"Combined CSV file '{combined_file_name}' has been created.")

    return f"Combined CSV file '{combined_file_name}' has been created."


In [11]:
test = ['world','test']
google_trends(test, 2023)

world:2023-01-01 2023-01-31
The request failed: Google returned a response with code 429
Trying again in 60 seconds.


KeyboardInterrupt: 

In [ ]:
test = pd.read_csv("test.csv")
test.head()

,Unnamed: 0


In [12]:
google_trends_2(djia_tickers, 2004)

The request failed: Google returned a response with code 429
Trying again in 60 seconds.


KeyboardInterrupt: 

In [ ]:
def fetch_google_trends_before2013(keyword, start_year):
    try:
        # Get the daily data
        data = dailydata.get_daily_data(keyword, start_year, 1, 2013, 12, geo='US')

        # Save the data to a CSV file
        data.to_csv('data.csv', index=False)

        # Check if 'isPartial' exists in the DataFrame
        if 'isPartial' in data.columns:
            data = data.drop(columns=['isPartial'])
    except KeyError as e:
        print(f"An error occurred: {e}")
        data = pd.DataFrame()

    # Sleep for 1 second to avoid hitting the rate limit
    time.sleep(6)

    return data


In [ ]:
fetch_google_trends_before2013("WBA", 2004)

The request failed: Google returned a response with code 429
Trying again in 60 seconds.


KeyboardInterrupt: 

### Checking the CSV files from the function as pandas dataframes

In [ ]:
#CHECKING IF CSV FILES DOWNLOADED PROPERLY 

df_AAPL = pd.read_csv("AAPL.csv", index_col=0)
df_AAPL

,AAPL
date,
2004-01-01,0.00
2004-01-02,0.00
2004-01-03,0.00
2004-01-04,0.00
2004-01-05,0.00
...,...
2023-12-27,19.20
2023-12-28,16.32
2023-12-29,14.40


Columns explained: 

1. AAPL_unscaled:
2. AAPL_Monthly
3. isPartial: This column indicates whether the data for the corresponding date is complete or partial. If the value is "True", it means the data for that date is incomplete and subject to revision. If the value is "False", it means the data is complete for that date.
4. scale
5. AAPL

The column named after the word argument contains the daily search volume already scaled and comparable through time. The column f'{word}_unscaled' is the original daily data fetched month by month, and it is not comparable across different months (but is comparable within a month).The column f'{word}_monthly' contains the original monthly datafetched at once. The values in this column have been backfilled so that there are no NaN present.The column 'scale' contains the scale used to obtain the scaleddaily data.

## Documentation:

### How is Google Trends data normalized?

Google Trends normalizes search data to make comparisons between terms easier. Search results are normalized to the time and location of a query by the following process:

Each data point is divided by the total searches of the geography and time range it represents to compare relative popularity. Otherwise, places with the most search volume would always be ranked highest.

The resulting numbers are then scaled on a range of 0 to 100 based on a topic’s proportion to all searches on all topics.

Different regions that show the same search interest for a term don't always have the same total search volumes.